# Pre-Proceso (1)


Validación de los datos
* Revisión de valores anómalos
* Revisión de valores faltantes
* Inconsistencias


Definir SparkContext y SQLContext

In [1]:
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SQLContext
sqlContext=SQLContext(sc)


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/22 11:07:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/22 11:07:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/22 11:07:20 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
/home/aldos/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
bd = sqlContext.read.format("com.databricks.spark.csv"
).option("header", "true"
).load("../On_Time_On_Time_Performance_2016_12.csv", 
       inferSchema=True)

In [3]:

bd=bd.select('Year','Month','DayofMonth','DayOfWeek','CRSDepTime','UniqueCarrier', 'TailNum', 'ArrDelay','DepDelay','Origin','Dest','Distance','Cancelled', 'Diverted','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay') 

In [4]:
bd2 = bd.filter( bd.Origin.isin('ATL','ORD','DEN','LAX','DFW','SFO','PHX','LAS') & bd.Dest.isin(
    'ATL','ORD','DEN','LAX','DFW','SFO','PHX','LAS') )

sqlContext.registerDataFrameAsTable(bd2, "bd2")

22/02/22 11:07:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## Validación

### Valores Anómalos

In [5]:
bd2.describe('ArrDelay').show()

+-------+------------------+
|summary|          ArrDelay|
+-------+------------------+
|  count|             30466|
|   mean|13.300695857677411|
| stddev| 53.50946738163739|
|    min|             -53.0|
|    max|            2028.0|
+-------+------------------+



In [6]:
bd2.describe().show()

+-------+------+-----+------------------+------------------+------------------+-------------+-------+------------------+-----------------+------+-----+-----------------+--------------------+--------------------+-----------------+------------------+------------------+--------------------+-----------------+
|summary|  Year|Month|        DayofMonth|         DayOfWeek|        CRSDepTime|UniqueCarrier|TailNum|          ArrDelay|         DepDelay|Origin| Dest|         Distance|           Cancelled|            Diverted|     CarrierDelay|      WeatherDelay|          NASDelay|       SecurityDelay|LateAircraftDelay|
+-------+------+-----+------------------+------------------+------------------+-------------+-------+------------------+-----------------+------+-----+-----------------+--------------------+--------------------+-----------------+------------------+------------------+--------------------+-----------------+
|  count| 30965|30965|             30965|             30965|             30965|

Exportamos la tabla a Pandas para mejorar la visualización

In [7]:
bd2.describe().toPandas().set_index('summary').transpose()

summary            count                  mean                stddev     min  \
Year               30965                2016.0                   0.0    2016   
Month              30965                  12.0                   0.0      12   
DayofMonth         30965    15.943484579363798     8.876012086360875       1   
DayOfWeek          30965     4.008719522041014    1.9259616915158768       1   
CRSDepTime         30965    1336.3809785241401    507.69693420285483       5   
UniqueCarrier      30965                  None                  None      AA   
TailNum            30820                  None                  None  N001AA   
ArrDelay           30466    13.300695857677411     53.50946738163739   -53.0   
DepDelay           30501     17.28562997934494     49.93901250742788   -21.0   
Origin             30965                  None                  None     ATL   
Dest               30965                  None                  None     ATL   
Distance           30965     932.5556596156952     547.1294895897895   236.0   
Cancelled          30965  0.015533667043436138   0.12366432835495896     0.0   
Diverted           30965  5.813014694009365E-4  0.024103574850036433     0.0   
CarrierDelay        8401     17.54874419711939      52.5742358244979     0.0   
WeatherDelay        8401     2.977978812046185    19.763653888123834     0.0   
NASDelay            8401    18.532674681585526     33.10933777220974     0.0   
SecurityDelay       8401  0.034638733484109036    0.8447964060560694     0.0   
LateAircraftDelay   8401     28.28437090822521     52.01295584698594     0.0   

summary               max  
Year                 2016  
Month                  12  
DayofMonth             31  
DayOfWeek               7  
CRSDepTime           2359  
UniqueCarrier          WN  
TailNum            N999DN  
ArrDelay           2028.0  
DepDelay           2040.0  
Origin                SFO  
Dest                  SFO  
Distance           2139.0  
Cancelled             1.0  
Diverted              1.0  
CarrierDelay       1276.0  
WeatherDelay        866.0  
NASDelay            407.0  
SecurityDelay        33.0  
LateAircraftDelay   806.0

### Inconsistencias

Variable Distance en función de los diferentes aeropuertos 

In [8]:
bd2.select('Origin','Dest','Distance').groupBy('Origin','Dest').mean().show()

+------+----+-------------+
|Origin|Dest|avg(Distance)|
+------+----+-------------+
|   LAS| ORD|       1514.0|
|   PHX| DFW|        868.0|
|   LAX| DFW|       1235.0|
|   PHX| ATL|       1587.0|
|   ATL| ORD|        606.0|
|   DFW| ORD|        802.0|
|   DEN| SFO|        967.0|
|   DFW| DEN|        641.0|
|   PHX| ORD|       1440.0|
|   SFO| ORD|       1846.0|
|   ATL| DEN|       1199.0|
|   SFO| DFW|       1464.0|
|   DEN| LAS|        628.0|
|   ATL| SFO|       2139.0|
|   ATL| LAS|       1747.0|
|   ORD| DEN|        888.0|
|   LAS| PHX|        255.0|
|   ATL| PHX|       1587.0|
|   SFO| DEN|        967.0|
|   PHX| LAS|        255.0|
+------+----+-------------+
only showing top 20 rows



In [9]:
sqlContext.sql("select Origin, Dest, count(*) as n, mean(Distance) as mean, \
                min(Distance) as min , max(Distance) as max,  \
                max(Distance) - min(Distance) as dif, \
                stddev(Distance) as stdev \
                from bd2 \
                group by Origin, Dest" ).show()

+------+----+---+------+------+------+---+--------------------+
|Origin|Dest|  n|  mean|   min|   max|dif|               stdev|
+------+----+---+------+------+------+---+--------------------+
|   LAS| ORD|302|1514.0|1514.0|1514.0|0.0|                 0.0|
|   PHX| DFW|380| 868.0| 868.0| 868.0|0.0|                 0.0|
|   LAX| DFW|531|1235.0|1235.0|1235.0|0.0|                 0.0|
|   PHX| ATL|312|1587.0|1587.0|1587.0|0.0|1.137196389861961...|
|   ATL| ORD|530| 606.0| 606.0| 606.0|0.0|                 0.0|
|   DFW| ORD|578| 802.0| 802.0| 802.0|0.0|                 0.0|
|   DEN| SFO|674| 967.0| 967.0| 967.0|0.0|                 0.0|
|   DFW| DEN|423| 641.0| 641.0| 641.0|0.0|                 0.0|
|   PHX| ORD|439|1440.0|1440.0|1440.0|0.0|                 0.0|
|   SFO| ORD|667|1846.0|1846.0|1846.0|0.0|                 0.0|
|   ATL| DEN|486|1199.0|1199.0|1199.0|0.0|                 0.0|
|   SFO| DFW|403|1464.0|1464.0|1464.0|0.0|                 0.0|
|   DEN| LAS|559| 628.0| 628.0| 628.0|0.

In [10]:
sqlContext.sql("select Origin, Dest, count(*) as n, mean(Distance) as mean, \
                min(Distance) as min , max(Distance) as max,  \
                max(Distance) - min(Distance) as dif, \
                stddev(Distance) as stdev \
                from bd2 \
                group by Origin, Dest" ).toPandas()

Origin Dest     n    mean     min     max  dif         stdev
0     LAS  ORD   302  1514.0  1514.0  1514.0  0.0  0.000000e+00
1     PHX  DFW   380   868.0   868.0   868.0  0.0  0.000000e+00
2     LAX  DFW   531  1235.0  1235.0  1235.0  0.0  0.000000e+00
3     PHX  ATL   312  1587.0  1587.0  1587.0  0.0  1.137196e-13
4     ATL  ORD   530   606.0   606.0   606.0  0.0  0.000000e+00
5     DFW  ORD   578   802.0   802.0   802.0  0.0  0.000000e+00
6     DEN  SFO   674   967.0   967.0   967.0  0.0  0.000000e+00
7     DFW  DEN   423   641.0   641.0   641.0  0.0  0.000000e+00
8     PHX  ORD   439  1440.0  1440.0  1440.0  0.0  0.000000e+00
9     SFO  ORD   667  1846.0  1846.0  1846.0  0.0  0.000000e+00
10    ATL  DEN   486  1199.0  1199.0  1199.0  0.0  0.000000e+00
11    SFO  DFW   403  1464.0  1464.0  1464.0  0.0  0.000000e+00
12    DEN  LAS   559   628.0   628.0   628.0  0.0  0.000000e+00
13    ATL  SFO   278  2139.0  2139.0  2139.0  0.0  0.000000e+00
14    ATL  LAS   368  1747.0  1747.0  1747.0  0.0  0.000000e+00
15    ORD  DEN   566   888.0   888.0   888.0  0.0  5.663417e-14
16    LAS  PHX   451   255.0   255.0   255.0  0.0  0.000000e+00
17    ATL  PHX   311  1587.0  1587.0  1587.0  0.0  0.000000e+00
18    SFO  DEN   670   967.0   967.0   967.0  0.0  5.649908e-14
19    PHX  LAS   457   255.0   255.0   255.0  0.0  0.000000e+00
20    ORD  ATL   541   606.0   606.0   606.0  0.0  0.000000e+00
21    LAX  PHX   605   370.0   370.0   370.0  0.0  0.000000e+00
22    LAS  DEN   560   628.0   628.0   628.0  0.0  0.000000e+00
23    SFO  LAS   689   414.0   414.0   414.0  0.0  0.000000e+00
24    DEN  LAX   609   862.0   862.0   862.0  0.0  0.000000e+00
25    SFO  ATL   280  2139.0  2139.0  2139.0  0.0  0.000000e+00
26    ORD  DFW   564   802.0   802.0   802.0  0.0  0.000000e+00
27    DFW  ATL   585   731.0   731.0   731.0  0.0  0.000000e+00
28    DFW  PHX   385   868.0   868.0   868.0  0.0  0.000000e+00
29    DEN  ORD   535   888.0   888.0   888.0  0.0  0.000000e+00
30    LAX  DEN   578   862.0   862.0   862.0  0.0  0.000000e+00
31    LAX  SFO  1285   337.0   337.0   337.0  0.0  0.000000e+00
32    LAS  ATL   366  1747.0  1747.0  1747.0  0.0  0.000000e+00
33    LAS  DFW   359  1055.0  1055.0  1055.0  0.0  0.000000e+00
34    DFW  LAS   360  1055.0  1055.0  1055.0  0.0  0.000000e+00
35    DEN  ATL   473  1199.0  1199.0  1199.0  0.0  0.000000e+00
36    PHX  DEN   684   602.0   602.0   602.0  0.0  0.000000e+00
37    DFW  LAX   531  1235.0  1235.0  1235.0  0.0  0.000000e+00
38    LAX  ATL   537  1946.0  1946.0  1946.0  0.0  0.000000e+00
39    LAS  LAX   998   236.0   236.0   236.0  0.0  0.000000e+00
40    ORD  PHX   439  1440.0  1440.0  1440.0  0.0  0.000000e+00
41    ATL  LAX   541  1946.0  1946.0  1946.0  0.0  0.000000e+00
42    LAS  SFO   692   414.0   414.0   414.0  0.0  0.000000e+00
43    ORD  SFO   666  1846.0  1846.0  1846.0  0.0  0.000000e+00
44    DFW  SFO   402  1464.0  1464.0  1464.0  0.0  0.000000e+00
45    PHX  LAX   597   370.0   370.0   370.0  0.0  2.834957e-14
46    DEN  DFW   453   641.0   641.0   641.0  0.0  0.000000e+00
47    SFO  LAX  1324   337.0   337.0   337.0  0.0  0.000000e+00
48    DEN  PHX   680   602.0   602.0   602.0  0.0  0.000000e+00
49    SFO  PHX   431   651.0   651.0   651.0  0.0  0.000000e+00
50    LAX  ORD   778  1744.0  1744.0  1744.0  0.0  0.000000e+00
51    LAX  LAS   994   236.0   236.0   236.0  0.0  0.000000e+00
52    ORD  LAS   299  1514.0  1514.0  1514.0  0.0  0.000000e+00
53    PHX  SFO   433   651.0   651.0   651.0  0.0  0.000000e+00
54    ATL  DFW   585   731.0   731.0   731.0  0.0  0.000000e+00
55    ORD  LAX   742  1744.0  1744.0  1744.0  0.0  0.000000e+00

### Valores Faltantes

In [11]:
bd2.describe().toPandas().transpose()[0]

summary              count
Year                 30965
Month                30965
DayofMonth           30965
DayOfWeek            30965
CRSDepTime           30965
UniqueCarrier        30965
TailNum              30820
ArrDelay             30466
DepDelay             30501
Origin               30965
Dest                 30965
Distance             30965
Cancelled            30965
Diverted             30965
CarrierDelay          8401
WeatherDelay          8401
NASDelay              8401
SecurityDelay         8401
LateAircraftDelay     8401
Name: 0, dtype: object

Valores faltantes en ArrDelay

In [12]:
bd2.select('ArrDelay').count()

30965

In [13]:
from pyspark.sql.functions import col
bd2.describe('Year','ArrDelay').filter(col("summary") == "count").collect()

[Row(summary='count', Year='30965', ArrDelay='30466')]

In [14]:
bd2.select('ArrDelay').count() - bd2.select('ArrDelay').dropna().count()

499

Los valores faltantes en ArrDelay son causados por vuelos cancelados o desviados

In [15]:
bd2.select('Cancelled','Diverted').groupBy().sum().show()

+--------------+-------------+
|sum(Cancelled)|sum(Diverted)|
+--------------+-------------+
|         481.0|         18.0|
+--------------+-------------+



In [16]:
bd3 = bd2.filter(bd2.Cancelled==0
).filter(bd2.Diverted==0)

In [17]:
bd3.describe().toPandas().transpose()[0]

summary              count
Year                 30466
Month                30466
DayofMonth           30466
DayOfWeek            30466
CRSDepTime           30466
UniqueCarrier        30466
TailNum              30466
ArrDelay             30466
DepDelay             30466
Origin               30466
Dest                 30466
Distance             30466
Cancelled            30466
Diverted             30466
CarrierDelay          8401
WeatherDelay          8401
NASDelay              8401
SecurityDelay         8401
LateAircraftDelay     8401
Name: 0, dtype: object

Valores faltantes en Tipos de *Delay*

In [18]:
bd3.filter(bd3.ArrDelay<0).select('ArrDelay','CarrierDelay','WeatherDelay',
                                  'NASDelay','SecurityDelay','LateAircraftDelay').show(20)

+--------+------------+------------+--------+-------------+-----------------+
|ArrDelay|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+--------+------------+------------+--------+-------------+-----------------+
|    -7.0|        null|        null|    null|         null|             null|
|    -3.0|        null|        null|    null|         null|             null|
|    -3.0|        null|        null|    null|         null|             null|
|    -2.0|        null|        null|    null|         null|             null|
|    -2.0|        null|        null|    null|         null|             null|
|    -6.0|        null|        null|    null|         null|             null|
|    -9.0|        null|        null|    null|         null|             null|
|    -2.0|        null|        null|    null|         null|             null|
|   -16.0|        null|        null|    null|         null|             null|
|   -21.0|        null|        null|    null|         null|     

In [19]:
bd3.filter(bd3.ArrDelay>=0).select('ArrDelay','CarrierDelay','WeatherDelay',
                                  'NASDelay','SecurityDelay','LateAircraftDelay').show(20)

+--------+------------+------------+--------+-------------+-----------------+
|ArrDelay|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+--------+------------+------------+--------+-------------+-----------------+
|     0.0|        null|        null|    null|         null|             null|
|     7.0|        null|        null|    null|         null|             null|
|    13.0|        null|        null|    null|         null|             null|
|     0.0|        null|        null|    null|         null|             null|
|     2.0|        null|        null|    null|         null|             null|
|     1.0|        null|        null|    null|         null|             null|
|    13.0|        null|        null|    null|         null|             null|
|    10.0|        null|        null|    null|         null|             null|
|     8.0|        null|        null|    null|         null|             null|
|    51.0|         0.0|         0.0|    51.0|          0.0|     

In [20]:
bd3.filter(bd3.ArrDelay==15).select('ArrDelay','CarrierDelay','WeatherDelay',
                                  'NASDelay','SecurityDelay','LateAircraftDelay').show(20)

+--------+------------+------------+--------+-------------+-----------------+
|ArrDelay|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+--------+------------+------------+--------+-------------+-----------------+
|    15.0|        15.0|         0.0|     0.0|          0.0|              0.0|
|    15.0|        15.0|         0.0|     0.0|          0.0|              0.0|
|    15.0|         0.0|         0.0|    15.0|          0.0|              0.0|
|    15.0|         0.0|         0.0|    15.0|          0.0|              0.0|
|    15.0|         0.0|         0.0|    15.0|          0.0|              0.0|
|    15.0|         0.0|         0.0|    15.0|          0.0|              0.0|
|    15.0|         0.0|         6.0|     9.0|          0.0|              0.0|
|    15.0|         0.0|         0.0|    15.0|          0.0|              0.0|
|    15.0|         4.0|         0.0|     5.0|          0.0|              6.0|
|    15.0|         0.0|         0.0|    11.0|          0.0|     

In [21]:
bd3.filter(bd3.ArrDelay>=15).select('ArrDelay','CarrierDelay','WeatherDelay',
                                  'NASDelay','SecurityDelay','LateAircraftDelay').count()

8401

In [22]:
bd4 = bd3.na.fill({'CarrierDelay':0, 'WeatherDelay':0,'NASDelay':0,'SecurityDelay':0, 'LateAircraftDelay':0})
#bd4 = bd3.na.fill(0)
#bd4 = bd3.na.drop()

In [23]:
bd4.describe().toPandas().transpose()[0]

summary              count
Year                 30466
Month                30466
DayofMonth           30466
DayOfWeek            30466
CRSDepTime           30466
UniqueCarrier        30466
TailNum              30466
ArrDelay             30466
DepDelay             30466
Origin               30466
Dest                 30466
Distance             30466
Cancelled            30466
Diverted             30466
CarrierDelay         30466
WeatherDelay         30466
NASDelay             30466
SecurityDelay        30466
LateAircraftDelay    30466
Name: 0, dtype: object